In [1]:
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import urllib.request
import requests
import pandas as pd
import re
import csv

def crawl_page(link=''):
    # req = urllib.request.Request(
    #         link, 
    #         data=None, 
    #         headers={
    #             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    #             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393'
    #         }
    # )
    # try:
    #     page= urllib.request.urlopen(req)
    #     return_code= page.getcode()
    #     soup_page= BeautifulSoup(page,'html.parser')
    #     return {'page':soup_page, 'status': return_code}
    # except HTTPError as err:
    #     if err.code == 404:
    #         return_code=404
    #         return {'page':'no page', 'status': return_code}
    #     else:
    #         raise
    try:
        response = requests.get(
            url='http://api.scraperapi.com', #scrapeapi
            params={
                'api_key': '56fa8295856477d20660fa8c5dcd2638',
                'url': link,  
            },
            
        )
        return_code= 200
        soup_page= BeautifulSoup(response.content,'html.parser')
        return {'page':soup_page, 'status': return_code}
    except requests.exceptions.HTTPError as err:
        if err.response.status_code == 404:
            return_code=404
            return {'page':'no page', 'status': return_code}
    

def write_csv(line=[], file=''):
    with open(file, 'a', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=' ',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(line)

def remove_row_csv():
    with open('temp_27.csv') as inp:
        for link in csv.reader(inp):
            count=0
            temp=link
            with open('used_estate.txt') as out:
                for estate in csv.reader(out):
                    if re.match("(.*)/"+estate[0],temp[1]) is not None:
                        count+=1
            if count==0:
                new_row= [link[0]+','+link[1]]
                write_csv(new_row,"temp_28.csv")

def create_urls(file=''):
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 1
        for row in csv_reader:
            new_row= [str(line_count)+','+row[0]]
            write_csv(new_row,'property_url_v2.csv')
            line_count+=1

    

In [18]:
class DataCollect():
    def __init__(self):
        self.__url_to_open='https://www.property24.com'
        self.__soup=crawl_page(self.__url_to_open)['page']
        self.__area_url=[]
        self.__list_url=[]
        self.__urls=pd.read_csv("temp_9_v5.csv",usecols= ['id','url']) 
    
    def collect_all_first_page_url(self):
        other_links= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all('a',class_='p24_bold')
        for link in other_links:
            self.__area_url.append(self.__url_to_open+link.get('href'))
        first_element_tag= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all(class_='row')[1]
        first_link=first_element_tag.find('a').get('href')
        temp_soup= crawl_page(self.__url_to_open+first_link)['page']
        parent_link= temp_soup.find(id='breadCrumbContainer').find_all('a')[1]
        self.__area_url.append(self.__url_to_open+parent_link.get('href'))
    
    def collect_all_url(self):
        self.collect_all_first_page_url()
        for link in self.__area_url:
            soup= crawl_page(link)['page']
            link_house_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("houses-for-sale")).get('href')
            link_apart_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("apartments-for-sale")).get('href')
            link_townhouse_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("townhouses-for-sale")).get('href')
            self.__list_url.append(link_house_tag)
            self.__list_url.append(link_apart_tag)
            self.__list_url.append(link_townhouse_tag)

    def get_property_data(self, link=''):
        data={}

        estate_id= link.split('/')[-1].split('?')[0]

        prop_soup_element= crawl_page(self.__url_to_open+link)

        
        if prop_soup_element['status'] !=200:
            print('heres')
            return False
        
        prop_soup= prop_soup_element['page']

        if prop_soup is None:
            print('here')
            return False

        #check if page not found
        check_page_not_found_msg= prop_soup.select_one('.p24_not_found')
        if check_page_not_found_msg is not None:
            print("====> Page not found !!")
            return False
        
        #check page expired
        check_page_expired = prop_soup.select_one('.p24_results.p24_expired')
        if check_page_expired is not None:
            print("====> Page expired !!")
            return False
        

        name= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('h1').get_text()
        data['name']=name

        price= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('div',class_="p24_price").get_text()
        data['price']=price

        province= link.split('/')[4]
        data['province']=province

        city= link.split('/')[3]
        data['city']=city


        is_adress= len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) >= 3
        if is_adress:
            if len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) == 3:
                index=-1
            else:
                index=3
            address= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")[index].get_text()
            data['address']=address
        else:
            address=  prop_soup.select_one('.p24_addressPropOverview').get_text()
            data['address']=address

        #property overview
        if type(prop_soup.select_one('.p24_propertyOverview')) is not None:
            prop_overview_panel= prop_soup.find('div',class_='p24_propertyOverview').find('div',class_='panel-body')
            prop_overview_row= prop_overview_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #rooms
        rooms_panel=prop_soup.select_one('#js_accordion_rooms')
        if rooms_panel is not None:
            prop_overview_row= rooms_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #external_features
        external_features_panel=prop_soup.select_one('#js_accordion_externalfeatures')
        if external_features_panel is not None:
            prop_overview_row= external_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value
        
        #other_features
        other_features_panel=prop_soup.select_one('#js_accordion_otherfeatures')
        if other_features_panel is not None:
            prop_overview_row= other_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #building_features
        building_features_panel=prop_soup.select_one('#js_accordion_building')
        if building_features_panel is not None:
            prop_overview_row= building_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #points_interest
        interest_soup= crawl_page(self.__url_to_open+'/ListingReadOnly/PointsOfInterestForListing?ListingNumber=P24-'+estate_id)
        # points_interest_panel=prop_soup.select_one('#accordian-points-of-interest')
        # print(points_interest_panel)
        if interest_soup['status'] == 200:
            categories= interest_soup['page'].select('.js_P24_POICategory')
            for cat in categories:
                interest= cat.select_one('.row > .col-6 > .p24_semibold')
                distances= cat.select('.row > .col-6.noPadding.p24_semibold')
                dist_tab=[]
                for dist in distances:
                    dist_tab.append(dist.get_text())
                data[interest.get_text()]=dist_tab

        print(str(data))
        write_csv([str(data)],'data_property_v2.csv')

        
        return True

    def crawl_property_per_page(self,city,page,number_properties):
        link=self.__url_to_open+city+'/p'+str(page)
        print('==>Page link: ',link)
        cur_soup= crawl_page(link)['page']
        properties= cur_soup.find_all('div',class_='js_resultTile')
        for prop_index in range(len(properties)-1):
            prop_link= properties[prop_index].find('a').get('href')
            write_csv([prop_link],'data_property_url.csv')

        number_properties+= len(properties)
        return number_properties

    def crawl_property_per_city(self,city,cur_page):
        soup= crawl_page(self.__url_to_open+city)['page']
        pagination= soup.find('div',class_='p24_pager').find_all('li')[-1].find('a').get('data-pagenumber')
        page_number= int(pagination)
        number_properties=0
        print("City url: ",city)
        for page in range(cur_page, page_number+1):
            print("=>Page number: ",page)
            number_properties = self.crawl_property_per_page(city,page,number_properties)
        print(number_properties)
    
    def crawl_properties(self):
        #shuffle
        self.__urls = self.__urls.sample(frac = 1).reset_index(drop=True)
        for i in range(self.__urls.size):
            prop_link= self.__urls.iloc[i]['url']
            
            print('==>Property link: ', prop_link)
            check_property_crawl = self.get_property_data(prop_link)
            if check_property_crawl:
                self.__urls= self.__urls.drop(self.__urls.index[i])
                self.__urls.to_csv('temp_10_v5.csv',index=False)

    def crawl_property_url(self):
        self.__list_url=['/townhouses-for-sale/gauteng/1']
        for link in self.__list_url:
            self.crawl_property_per_city(link,1)

In [19]:
dc= DataCollect()
try:
    dc.crawl_properties()
except:
    dc.crawl_properties()

==>Property link:  /for-sale/melrose-arch/johannesburg/gauteng/10386/109463896
{'name': '0.5 Bedroom Apartment / Flat for Sale in Melrose Arch', 'price': 'R 1 750 000', 'province': 'gauteng', 'city': 'johannesburg', 'address': '\n0.5 Bedroom Apartment / Flat for Sale in Melrose Arch\n', 'Listing Number': '109463896', 'Type of Property': 'Apartment / Flat', 'Street Address': '1 Whiteley Road, Melrose Arch', 'Listing Date': '21 December 2020', 'Floor Size': '44 m²', 'Price per m²': 'R 39 773', 'Levies': 'R 1 944', 'Pets Allowed': 'No', 'Bedroom': '0.5', 'Bathrooms': '1', 'Kitchens': '1', 'Lounges': '1', 'Parking': '1', 'Pool': 'Yes', 'Food and Entertainment': ['0.30km', '0.31km', '0.32km', '0.34km', '1.78km', '1.88km', '1.88km', '2.21km', '2.37km', '2.38km'], 'Education': ['0.50km', '1.27km', '1.54km', '1.57km', '1.65km', '1.76km', '2.07km', '2.12km', '2.22km', '2.29km', '2.37km'], 'Sports and Leisure': ['0.84km', '0.89km'], 'Transport and Public Services': ['1.68km', '1.70km', '1.74km',

====> Page not found !!
==>Property link:  /for-sale/fourways-gardens/sandton/gauteng/5857/109269766
{'name': '3 Bedroom House for Sale in Fourways Gardens', 'price': 'R 3 199 000', 'province': 'gauteng', 'city': 'sandton', 'address': '\n3 Bedroom House for Sale in Fourways Gardens\n', 'Listing Number': '109269766', 'Type of Property': 'House', 'Street Address': '6 Park Way, Fourways Gardens', 'Listing Date': '23 October 2020', 'Erf Size': '1 157 m²', 'Levies': 'R 2 672', 'Pets Allowed': 'Yes', 'Bedrooms': '3', 'Bathrooms': '2', 'Kitchens': '1', 'Lounges': '2', 'Dining Rooms': '1', 'Garage': '2', 'Garden': 'Yes', 'Pool': 'Yes', 'Transport and Public Services': ['0.29km', '1.39km'], 'Shopping': ['0.30km', '0.30km', '0.31km', '1.52km', '1.63km'], 'Food and Entertainment': ['0.31km', '0.32km', '0.32km', '0.34km', '1.39km', '1.41km', '1.45km', '1.52km', '1.55km', '1.58km', '1.59km', '1.60km', '1.69km', '1.73km', '1.75km', '1.75km'], 'Health': ['1.00km', '1.01km', '1.26km', '1.60km'], 'Educ

{'name': '4 Bedroom House for Sale in Wavecrest', 'price': 'R 10 500 000', 'province': 'eastern-cape', 'city': 'jeffreys-bay', 'address': 'Wavecrest, Jeffreys Bay', 'Listing Number': '107378167', 'Type of Property': 'House', 'Description': 'Double Storey', 'Listing Date': '05 April 2019', 'Erf Size': '630 m²', 'Floor Size': '315 m²', 'Pets Allowed': 'Yes', 'Coverage': '50 %', 'Bedrooms': '4', 'Bathrooms': '4', 'Kitchen': 'Scullery, Laundry, Eye Level Oven, Granite Tops', 'Domestic Rooms': '1', 'Domestic Bathrooms': '1', 'Lounges': '1', 'Family/TV Rooms': '1', 'Braai Rooms': '1', 'Entrance Halls': '1', 'Other': 'Covered Patio. Balcony.', 'Garage': '2', 'Gardens': '1', 'Outbuilding': 'Staff Quarters/Domestic Rooms, Domestic Bathroom', 'Security': 'Trellidoor.', 'Special Features': 'Water Filter.', 'Roof': 'Tile', 'Wall': 'Face-brick', 'Window': 'Aluminium', 'Transport and Public Services': ['2.43km', '2.87km', '7.81km', '9.59km', '9.81km'], 'Education': ['3.37km', '4.27km', '4.34km', '4.

====> Page expired !!
==>Property link:  /for-sale/schoemansville/hartbeespoort/north-west/3061/109707016
{'name': '5 Bedroom Townhouse for Sale in Schoemansville', 'price': 'R 2 575 000', 'province': 'north-west', 'city': 'hartbeespoort', 'address': '\n5 Bedroom Townhouse for Sale in Schoemansville\n', 'Listing Number': '109707016', 'Type of Property': 'Townhouse', 'Street Address': ' 43 St Monica Street, Schoemansville', 'Description': 'Double Storey', 'Lifestyle': 'Complex', 'Listing Date': '03 March 2021', 'Erf Size': '1 488 m²', 'Floor Size': '305 m²', 'Levies': 'R 2 250', 'Rates and Taxes': 'R 1 300', 'Pets Allowed': 'Yes', 'Coverage': '20 %', 'Bedrooms': '5', 'Bedroom 1': 'Carpets, Tiled Floors, Curtain Rails, Open Plan, Built-in Cupboards', 'Bedroom 2': 'Tiled Floors, Curtain Rails, Built-in Cupboards', 'Bedroom 3': 'Tiled Floors, Curtain Rails', 'Bedroom 4': 'Tiled Floors, Curtain Rails', 'Bedroom 5': 'Tiled Floors, Curtain Rails', 'Bathrooms': '4', 'Bathroom 1': 'Tiled Floors

{'name': '3 Bedroom Townhouse for Sale in Baillie Park', 'price': 'R 1 395 000', 'province': 'north-west', 'city': 'potchefstroom', 'address': 'Baillie Park, Potchefstroom', 'Listing Number': '110257497', 'Type of Property': 'Townhouse', 'Listing Date': '02 August 2021', 'Floor Size': '130 m²', 'Levies': 'R 1 584', 'Rates and Taxes': 'R 250', 'Bedrooms': '3', 'Bedroom 1': 'Built-in Cupboards', 'Bedroom 2': 'Built-in Cupboards', 'Bedroom 3': 'Built-in Cupboards', 'Bathrooms': '2', 'Kitchen': 'Laundry', 'Lounge': 'Patio', 'Dining Rooms': '1', 'Family/TV Rooms': '1', 'Garage': '2', 'Parking': '2', 'Parking 1': 'Carport', 'Parking 2': 'Carport', 'Gardens': '1', 'Outbuilding': 'braai room', 'Security': 'Totally Fenced, Electric Garage, Electric Gate, Alarm System, Burglar Bars', 'Special Features': 'Pre-paid electricity', 'Roof': 'Tile', 'Window': 'Steel ', 'Education': ['0.53km', '1.00km', '1.52km', '1.66km', '2.53km', '2.74km', '2.79km', '2.80km', '2.86km', '2.91km', '2.94km', '3.04km', '

{'name': '2 Bedroom Townhouse for Sale in Royal Ascot', 'price': 'R 1 650 000', 'province': 'western-cape', 'city': 'milnerton', 'address': '\n2 Bedroom Townhouse for Sale in Royal Ascot\n', 'Listing Number': '109443187', 'Type of Property': 'Townhouse', 'Street Address': ' Sea Cottage Way,  Ascot Village, Royal Ascot', 'Lifestyle': 'Estate, Security Complex, Security Estate', 'Listing Date': '11 December 2020', 'Floor Size': '70 m²', 'Levies': 'R 1 750', 'Rates and Taxes': 'R 550', 'Bedrooms': '2', 'Bedroom 1': 'Tiled Floors, Blinds, Built-in Cupboards', 'Bedroom 2': 'Tiled Floors, Blinds, TV Port, Built-in Cupboards', 'Bathrooms': '2', 'Bathroom 1': 'Tiled Floors, Basin, Bath, Toilet', 'Bathroom 2': 'Tiled Floors, Basin, Shower, En suite, Toilet', 'Kitchen': 'Breakfast Nook, Stove (Oven & Hob), Granite Tops, Washing Machine Connection, Tiled Floors, Open Plan, Centre Island', 'Lounge': 'Tiled Floors, Balcony, Blinds, TV Port, Open Plan', 'Parking': '2', 'Parking 1': 'Secure Parking',

{'name': '3 Bedroom Apartment / Flat for Sale in Montclair', 'price': 'R 1 195 000', 'province': 'kwazulu-natal', 'city': 'durban', 'address': 'Montclair, Durban', 'Listing Number': '110233308', 'Type of Property': 'Apartment / Flat', 'Listing Date': '26 July 2021', 'Floor Size': '108 m²', 'Price per m²': 'R 11 065', 'Bedrooms': '3', 'Bathrooms': '2', 'Kitchen': 'Scullery, Extractor Fan, Granite Tops, Tiled Floors, Built-in Cupboards, Under Counter Oven, Hob', 'Lounges': '1', 'Dining Rooms': '1', 'Garage': '1', 'Parking': 'Visitors Parking', 'Pools': '1', 'Security': 'Totally Fenced, Intercom, Burglar Bars', 'Temperature Control': 'Air Conditioning Unit, Cooling Fans', 'Education': ['0.13km', '0.22km', '0.28km', '1.14km', '1.15km', '1.15km', '1.26km', '1.55km', '1.70km', '1.72km', '1.78km', '1.98km', '2.01km', '2.26km', '2.65km', '2.68km', '2.69km', '2.73km', '2.79km', '2.80km', '2.93km', '3.05km', '3.19km', '3.19km'], 'Transport and Public Services': ['0.94km', '1.18km', '2.43km', '3.

AttributeError: 'NoneType' object has no attribute 'find'